# Motor Control

**Note: The wiring for this is very important.  Please double-check your connections before proceeding.**

In [2]:
# Import GPIO Libraries
import RPi.GPIO as GPIO
import time
#GPIO.cleanup()

In [3]:
GPIO.setmode(GPIO.BCM)

coil_A_1_pin = 12
coil_A_2_pin = 16
coil_B_1_pin = 20
coil_B_2_pin = 21

GPIO.setup(coil_A_1_pin, GPIO.OUT)
GPIO.setup(coil_A_2_pin, GPIO.OUT)
GPIO.setup(coil_B_1_pin, GPIO.OUT)
GPIO.setup(coil_B_2_pin, GPIO.OUT)

#forward_seq = ['1001', '1100', '0110', '0011']
#forward_seq = ['1000', '0100', '0010', '0001']
#forward_seq = ['1001', '1100', '1010', '0110', '0101', '0011']
#forward_seq = ['1001', '1100','0110','0011']
forward_seq = ['1100', '0100', '0110', '0010', '0011', '0001', '1001', '1000']
reverse_seq = list(forward_seq) # to copy the list
reverse_seq.reverse()


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  # This is added back by InteractiveShellApp.init_path()


In [4]:
last_step='0000'

def forward(delay, steps):
  for i in range(steps):
    for s in forward_seq:
      set_step(s)
      time.sleep(delay)

def backwards(delay, steps):
  for i in range(steps):
    for s in reverse_seq:
      set_step(s)
      time.sleep(delay)


def set_step(step):
  GPIO.output(coil_A_1_pin, step[0] == '1')
  GPIO.output(coil_A_2_pin, step[1] == '1')
  GPIO.output(coil_B_1_pin, step[2] == '1')
  GPIO.output(coil_B_2_pin, step[3] == '1')
  



In [5]:
set_step('0000')
delay = 2
# Time Delay (ms)
steps = 25
forward(int(delay) / 1000.0, int(steps))

#backwards(int(delay) / 1000.0, int(steps))
#set_step('0000')

# Servo Control

In [202]:
set_step('0000')

In [213]:
GPIO.setmode(GPIO.BCM)
GPIO.setup(18, GPIO.OUT)
p = GPIO.PWM(18, 50)

In [214]:
p.start(7.5)

In [215]:
p.ChangeDutyCycle(6.4)

In [395]:
delay = 4
start_pos = 5.0
hold_pos = 6.45
push_pos = 8.3

## Rotate whole cube:

In [485]:
def servo_slow_move(servo_obj, start_position, end_position, increments=10, delay=0.05):
    # try to s_l_o_w_l_y move the servo by putting a delay between small incremental moves
    for i in range(increments):
        increm = float(increments)
        move_percent = (increm-i)/increm -(1.0/increm)  # try to end with 0 percent
        servo_obj.ChangeDutyCycle((move_percent*(start_position - end_position))+end_position)
        time.sleep(delay)  # this is the slight delay
        #print move_percent

In [486]:
servo_slow_move(p, start_pos, hold_pos, delay=.1)

In [501]:
def rotate_cube(num_turns):
    for i in range(num_turns):
        p.ChangeDutyCycle(start_pos)
        time.sleep(0.25)
        servo_slow_move(p, start_pos, push_pos, increments=2, delay=0.1)
        time.sleep(0.8)
        servo_slow_move(p, push_pos, start_pos, increments=10, delay=0.1)
        #p.ChangeDutyCycle(start_pos)
        time.sleep(0.2)

In [504]:
def hold_cube():
    # Grab the cube (or at least hold it still on top)
    time.sleep(0.2)
    servo_slow_move(p, start_pos, hold_pos, increments=10, delay=0.1)
    time.sleep(0.2)

In [505]:
rotate_cube(2)

In [507]:
hold_cube()

In [511]:
p??

## Now try a turn:

In [6]:
def rotate_side(num_turns):
    """ rotates the bottom face a quarter turn for each num_turns
    """
    
    hold_cube()
    
    delay = 3  # Motor Time Delay (ms)
    steps = 13 # a quarter turn(ish)
    
    for i in range(num_turns):
        adj = i % 2
        forward(int(delay) / 1000.0, int(steps-adj))
        #print "rotate steps: %s, adj: %s" % (steps, adj)
        time.sleep(.25)
        backwards(delay/1000.0, 1)
        set_step('1001')
    
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.3)

In [7]:
def straighten_cube(wiggle=2):
    # Grab the cube (or at least hold it still on top)
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.4)
    p.ChangeDutyCycle((hold_pos+start_pos)/2.0)  # move halfway to slow things down
    time.sleep(0.2)
    p.ChangeDutyCycle(hold_pos)
    time.sleep(0.5)
    
    forward(int(delay) / 1000.0, 2)
    #print "rotate steps: %s, adj: %s" % (steps, adj)
    time.sleep(.25)
    backwards(delay/1000.0, 2)
    set_step('1001')
    
    time.sleep(0.2)
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.4)

In [8]:
#rotate_side(1)
straighten_cube(4)

NameError: global name 'p' is not defined

In [353]:
def moves_seq(move_str):
    moves = move_str.split(' ')
    for move in moves:
        if move=='c':
            rotate_cube(1)
        elif move=='s':
            rotate_side(1)
            straighten_cube()
            

In [382]:
moves_seq('c s c s s c s s')

In [338]:
set_step('1001')

In [337]:
set_step('0000')

In [53]:
p.stop()
GPIO.cleanup()